If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. It also returns an image ROI which can be used to crop the result.

The calibration photos are displayed to screen and the matrix and distoran vector is saved to a pickle data file

In [33]:
import cv2
import numpy as np
import glob
from datetime import datetime
import pickle

cameraSerial = 'goproHero4sessionC3141324866724'
filename = cameraSerial + '.dat'
alpha = 1

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')
print(images)

['cal_1.jpg', 'cal_2.jpg', 'cal_3.jpg', 'cal_4.jpg', 'cal_5.jpg', 'cal_6.jpg', 'cal_7.jpg']


In [13]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
img = cv2.imread(images[0])
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),alpha,(w,h))


In [31]:
data = {"camera": cameraSerial, "camera matrix": list(newcameramtx), "dist_coeff": roi}
data

{'camera': 'goproHero4sessionC3141324866724',
 'camera matrix': [array([ 1493.34301758,     0.        ,   878.49371629]),
  array([   0.        ,  800.93096924,  568.50357481]),
  array([ 0.,  0.,  1.])],
 'dist_coeff': (797, 553, 131, 28)}

In [34]:
with open(filename, 'w') as f:
    pickle.dump(data, f)
    

In [25]:
newcameramtx

array([[  1.49334302e+03,   0.00000000e+00,   8.78493716e+02],
       [  0.00000000e+00,   8.00930969e+02,   5.68503575e+02],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])